# Блок №2

В следующих задачах продолжим работать с данными мобильного приложения крупной розничной сети. В этих данных отражено то, как наше приложение устанавливают (`installs`), как активно в нем просматривают товары (`events`), как активно в нем покупают (`checks`), и данные о склейке устройств с логинами (`devices`).

Note: Далее для проверки задач используются сэмплы (кусочки) данных, поэтому результирующие таблицы на LMS могут немного отличаться от тех, что вы получите в ClickHouse. Ответы на текстовые шаги или задачи с вводом ответа основаны на таблицах в ClickHouse.

Теперь описание данных:

`installs` — содержит данные об установках приложения по дням.

+ DeviceID — идентификатор устройства, на которое было установлено приложение;
+ InstallationDate — дата установки приложения;
+ InstallCost — цена установки приложения в рублях;
+ Platform — платформа, на которой было установлено приложение (iOS/ Android);
+ Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).

`events` — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

+ DeviceID — идентификатор устройства, на котором используется приложение;
+ AppPlatform — платформа, на которой используется приложение (iOS/ Android);
+ EventDate — дата, за которую собрана статистика;
+ events — количество просмотров всех товаров за этот день у этого DeviceID.

`checks` — содержит данные о покупках пользователей в приложении по дням

+ UserID — идентификатор пользователя;
+ Rub — суммарный чек пользователя на дату;
+ BuyDate — дата, за которую собрана статистика.

Особенность приложения заключается в том, что для просмотра товаров не обязательно авторизовываться. До момента авторизации про пользователя известен только его `DeviceID` — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается `UserID`, и тогда мы уже знаем два его идентификатора: `DeviceID` (устройство) и `UserID` (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только `DeviceID`. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с `UserID`. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия `DeviceID` к `UserID`, то есть таблица `devices`:

+ DeviceID — идентификатор устройства;
+ UserID — идентификатор пользователя.

# Задание 1
___
Таблица `tickets` включает следующие поля:

+ `date` – дата продажи
+ `id` – идентификатор чека
+ `manager` – менеджер
+ `revenue` – сумма чека

Выберите некорректные конструкции запросов. Внимательно читайте варианты ответов: при каждой попытке список вариантов может меняться!

Чтобы отобрать некорректные формы запросов, нужно помнить о следующих правилах:

1) группировка (GROUP BY) не может проводиться по той же колонке, к которой применяется агрегирующая функция;
2) при применении оператора HAVING необходимо указывать алиас колонки, к которой ранее в блоке SELECT была применена агрегирующая функция, а не проводить агрегацию повторно;
3) оператор WHERE фильтрует строки таблицы до агрегации.

Ответы:
```
SELECT manager, SUM(revenue) as SumRev FROM tickets GROUP BY revenue;

SELECT manager, id, MAX(revenue) as MaxRev FROM tickets GROUP BY manager HAVING MAX(revenue)>5000;

SELECT date, manager, AVG(revenue) as AvgRev FROM tickets GROUP BY date, manager WHERE AVG(revenue)>1000;
```

# Задание 2
___
В чем заключается главное отличие HAVING от WHERE? Выберите верные утверждения.

Главное правило - помнить, что HAVING напрямую связан с агрегацией и проводится после неё, в отличие от WHERE.

Ответы:
+ HAVING можно использовать без агрегации
+ WHERE выполняется перед агрегацией/группировкой, HAVING – после
+ WHERE можно использовать без агрегации

# Задание 3
___
Продолжим работать с таблицей `checks`.

Посчитайте, сколько покупок приходится на каждого клиента, указав имя столбца как `NumChecks`. Результирующую таблицу с `UserID`, `NumChecks` отсортируйте по убыванию (DESC) нового столбца. Ограничение вывода – 10.

Ответ:
```
SELECT 
    UserID,
    COUNT(Rub) AS NumChecks
FROM 
    checks
GROUP BY 
    UserID
ORDER BY 
    NumChecks DESC
LIMIT 10
```

# Задание 4
___
Давайте немного усложним задачу, и посчитаем ещё и сумму сделанных покупок в рублях (`Rub`).

Выведите в результирующую таблицу `UserID` клиентов, количество сделанных покупок (`NumChecks`) и сумму потраченных денег (`Revenue`). Результат отсортируйте по убыванию `Revenue`, оставив 10 записей.

Ответ:
```
SELECT 
    UserID, 
    COUNT(Rub) AS NumChecks,
    SUM(Rub) AS Revenue
FROM 
    checks
GROUP BY 
    UserID
ORDER BY 
    Revenue DESC
LIMIT 10
```

# Задание 5
___
Теперь давайте попробуем с помощью группировки и агрегатных функций посмотреть динамику трат клиентов. Для этого нужно посчитать по дням минимальный, средний, и максимальный чек. Иными словами, необходимо сгруппироваться по датам, и использовать агрегатные функции `MIN`, `AVG`, `MAX`.

Выведите минимальный (`MinCheck`), максимальный (`MaxCheck`) и средний (`AvgCheck`) чек по дням. В результирующей таблице используйте сортировку по убыванию (DESC) столбца `BuyDate`. Как и на предыдущем шаге, ограничение на вывод равно 10.

Ответ:
```
SELECT 
     BuyDate,
     MIN(Rub) MinCheck,
     MAX(Rub) MaxCheck,
     AVG(Rub) AvgCheck
FROM 
    checks
GROUP BY 
    BuyDate
ORDER BY 
    BuyDate DESC
LIMIT 10
```

# Задание 6
___
Представим, что для проведения новой акции нам потребовалось выгрузить сегмент пользователей, которые приносят наибольшую выручку.

Найдите покупателей, сумма покупок которых превышает 10 000 рублей (`Rub`). Для этого сгруппируйте пользователей по `UserID`, затем примените нужную агрегатную функцию и назовите новую колонку `Revenue`. Отфильтруйте нужные значения, результат отсортируйте по убыванию `UserID` и выведите 10 строк.

Ответ:
```
SELECT 
    UserID,
    SUM(Rub) AS Revenue
FROM 
    checks 
GROUP BY 
    UserID
HAVING 
    Revenue > 10000
ORDER BY 
    UserID DESC
LIMIT 10
```

# Задание 7
___
В этой и последующих задачах мы будем работать с таблицей `retail` в ClickHouse, которая содержит данные о покупках. 

Задания выполняются в ClickHouse, в качестве ответа, как правило, нужно будет вписывать значения какого-либо поля из результирующей таблицы.

Таблица включает следующие поля: 

+ InvoiceNo - идентификатор покупки
+ StockCode - идентификатор товара на складе
+ Description - текстовое описание товара
+ Quantity - количество товара
+ InvoiceDate - дата покупки
+ UnitPrice - цена товара
+ CustomerID - идентификатор покупателя
+ Country - страна

Представим, что к вам пришел менеджер с проблемой: за последний месяц он наблюдает снижение выручки в некоторых регионах, и нужно определить возможную причину этого снижения. Для начала можно сгруппировать данные о выручке по странам и найти среди них топ-5 стран по величине выручки, так как интереснее всего нам будет смотреть именно на данные этих стран.

Посчитайте выручку (`Revenue`) по странам и отсортируйте таблицу в порядке убывания выручки. В качестве ответа скопируйте название страны с самой высокой выручкой.  Обратите внимание, что в изначальной таблице указана цена одной единицы товара (`UnitPrice`) и количество купленного товара (`Quantity`), а не выручка, и для получения `Revenue` нужно будет использовать агрегирующую функцию `Sum` и оператор умножения `*`.

Правильный запрос:
```
SELECT
    SUM(UnitPrice * Quantity) AS Revenue,
    Country
FROM 
    retail
GROUP BY 
    Country
ORDER BY 
    Revenue DESC
```
Ответ:
```
United Kingdom
```

# Задание 8
____
В качестве более подробного описания выручки по странам можно посмотреть, каково среднее количество купленных товаров и средняя цена товара в покупках, совершенных в определенной стране.

Посчитайте среднее число купленных товаров по стране и среднюю цену товара, сгруппировав данные по странам и используя агрегирующую функцию `Avg`, и отсортируйте по убыванию средней цены товара (DESC).

Обратите внимание, что в данных встречаются строки с `Description` 'Manual', которые включают данные об удаленных из чека позициях. Для получения правильного ответа такие строки необходимо отфильтровать с использованием операторов `WHERE` и неравенства `!=`.

В качестве ответа впишите название страны, в которой самая высокая средняя цена за купленный товар.

Правильный запрос:
```
SELECT 
    AVG(UnitPrice) AS average_price,
    Country
FROM 
    retail
WHERE
    Description!='Manual'
GROUP BY 
    Country
ORDER BY 
    average_price DESC
```
Ответ:
```
Canada
```

# Задание 9
___
Теперь посмотрим на динамику общей суммы выручки по месяцам.

Посчитайте выручку (`Revenue`) по месяцам (обратите внимание, что в таблице указана цена одной единицы товара и количество купленного товара), округлив `InvoiceDate` к началу месяца с помощью `toStartOfMonth()`.

В качестве ответа укажите сумму выручки (`Revenue`) из первой строки результирующей таблицы, используя сортировку по убыванию (DESC) по столбцу `Revenue`. Не забывайте отфильтровать строки по условию `Description != 'Manual'`.

Правильный запрос:
```
SELECT 
    SUM(UnitPrice * Quantity) AS revenue,
    toStartOfMonth(InvoiceDate) AS rev_month
FROM 
    retail
WHERE 
    Description!='Manual'
GROUP BY 
    rev_month
ORDER BY 
    revenue DESC
```
Ответ:
```
1130664.25
```

# Задание 10
___
Давайте посмотрим на динамику выручки от покупателей, которые в среднем покупают самые дорогие товары.  В качестве целевой метрики будем использовать среднюю цену купленного товара (`UnitPrice`), данные посмотрим за март 2011 года.

Посчитайте среднюю цену товара клиента (`UnitPrice`) с группировкой по id покупателя, используя агрегирующую функцию `Avg`, затем выведите строки с покупками тех покупателей, у которых самая высокая средняя цена купленного товара в марте. Не забывайте отфильтровать строки по условию `Description != 'Manual'`. В качестве ответа впишите `CustomerID` покупателя с самой высокой средней ценой купленного товара.

Правильный запрос:
```
SELECT 
    AVG(UnitPrice) AS avg_price,
    CustomerID
FROM 
    retail
WHERE 
    Description!='Manual' and toStartOfMonth(InvoiceDate) = '2011-03-01'
GROUP BY 
    CustomerID
ORDER BY 
    avg_price DESC
LIMIT 1
```
Ответ:
```
13452
```

# Задание 11
___
Давайте попробуем ответить на следующий вопрос: "Как изменилось среднее, минимальное и максимальное количество купленного товара в стране с наибольшей выручкой в течение последних месяцев?"

Для этого нужно сгруппировать данные по странам и месяцам, посчитать среднее (`Avg`), минимальное (`Min`) и максимальное (`Max`) количество купленных товаров (`Quantity`), и отсортировать данные по месяцу, применить фильтр по стране (нам нужны данные из страны с наибольшей выручкой из задания 8 (step 8)). Если обратить внимание на полученное значение минимального количества товаров, то станет понятно, что там есть отрицательные значения, которые следует отфильтровать (также не забывайте отфильтровывать значения с `Description != 'Manual'`).

В качестве ответа скопируйте наибольшее историческое значение среднего числа купленных товаров для этой страны.

Так как при первичном просмотре данных, было обнаружено, что минимальное значение отрицательно, то есть присутствуют некоректные данные, которые повлияют на результат, и такие значения нужно отфильтровывать для получения корректного среднего значение.

Правильный запрос:
```
SELECT 
    toStartOfMonth(InvoiceDate) as month,
    AVG(Quantity) as avg_q,
    MIN(Quantity) as min_q,
    MAX(Quantity) as max_q
FROM 
    retail
WHERE 
    Description!='Manual' and Country = 'United Kingdom' and Quantity>0
GROUP BY 
    month
ORDER BY 
    avg_q DESC
LIMIT 1
```
Ответ:
```
16.44956626353448
```